# Latent Dirichlet Allocation

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
import random
from wordcloud import WordCloud 

In [2]:
data = pd.read_csv('npr.csv')
data.rename(columns={'Article':'article'},inplace=True)

In [3]:
data['article']=data['article'].fillna('')

Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [5]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [6]:
dtm = cv.fit_transform(data['article'])

## LDA

In [7]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
LDA = LatentDirichletAllocation(n_components=7,random_state=42)

In [9]:
LDA.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Showing Stored Words

In [10]:
len(cv.get_feature_names_out())

54777

In [11]:
for i in range(10):
    random_word_id = random.randint(0,6870)
    print(cv.get_feature_names_out()[random_word_id])

angie
bobbi
10s
archdiocese
accompanists
austerity
bootlegged
adaptable
amorphous
backfired


In [12]:
for i in range(10):
    random_word_id = random.randint(0,6870)
    print(cv.get_feature_names_out()[random_word_id])

bazaars
bellyaching
benefactors
arpad
bedding
beggar
bewitched
afrobeat
astrobiologists
asymptomatic


### Showing Top Words Per Topic

In [13]:
len(LDA.components_)

7

In [14]:
LDA.components_

array([[8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
        1.43006821e-01, 1.42902042e-01, 1.42861626e-01],
       [2.76191749e+01, 5.36394437e+02, 1.42857148e-01, ...,
        1.42861973e-01, 1.42857147e-01, 1.42906875e-01],
       [7.22783888e+00, 8.24033986e+02, 1.42857148e-01, ...,
        6.14236247e+00, 2.14061364e+00, 1.42923753e-01],
       ...,
       [3.11488651e+00, 3.50409655e+02, 1.42857147e-01, ...,
        1.42859912e-01, 1.42857146e-01, 1.42866614e-01],
       [4.61486388e+01, 5.14408600e+01, 3.14281373e+00, ...,
        1.43107628e-01, 1.43902481e-01, 2.14271779e+00],
       [4.93991422e-01, 4.18841042e+02, 1.42857151e-01, ...,
        1.42857146e-01, 1.43760101e-01, 1.42866201e-01]])

In [15]:
len(LDA.components_[0])

54777

In [16]:
single_topic = LDA.components_[0]

In [17]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([ 2475, 18302, 35285, ..., 22673, 42561, 42993])

In [18]:
# Word least representative of this topic
single_topic[6505]

48.49748714649684

In [19]:
# Word most representative of this topic
single_topic[2565]

0.14406508124639014

In [20]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993])

In [21]:
top_word_indices = single_topic.argsort()[-10:]

In [22]:
for index in top_word_indices:
    print(cv.get_feature_names_out()[index])

new
percent
government
company
million
care
people
health
said
says


In [23]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR THE TOPIC #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR THE TOPIC #0
['companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


THE TOP 15 WORDS FOR THE TOPIC #1
['military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


THE TOP 15 WORDS FOR THE TOPIC #2
['way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


THE TOP 15 WORDS FOR THE TOPIC #3
['time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


THE TOP 15 WORDS FOR THE TOPIC #4
['voters', 'vote', 'election', 'party', 'new', 'obama', 'court', 'republican', 'campaign', 'people', 'state', 'president', 'clinton', 'said', 'trump']


THE TOP 15 WORDS FOR THE TOPIC #5
['years', 'going', 've', 'life', 'don', 'new', 'way', 'music', 'really', 

### Attaching Discovered Topic Labels to Original Articles

In [24]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [25]:
dtm.shape

(11992, 54777)

In [26]:
len(data)

11992

In [27]:
topic_results = LDA.transform(dtm)

In [28]:
topic_results.shape

(11992, 7)

In [29]:
topic_results[0]

array([1.61040465e-02, 6.83341493e-01, 2.25376318e-04, 2.25369288e-04,
       2.99652737e-01, 2.25479379e-04, 2.25497980e-04])

In [30]:
topic_results[0].round(2)

array([0.02, 0.68, 0.  , 0.  , 0.3 , 0.  , 0.  ])

In [31]:
topic_results[0].argmax()

1

This means that our model thinks that the first article, number 0 belongs to topic 1.

### Combining with Original Data

In [32]:
data.head()

,article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [33]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 3, 4, 0])

In [34]:
data['predicted_topic'] = topic_results.argmax(axis=1)

In [35]:
data.head(5)

,article,predicted_topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
4,"From photography, illustration and video, to d...",2


#### Given that this dataset lacks labels, our attention won't be directed toward deciphering the specific meanings behind these numerical values. Instead, we will center our analysis around the cluster numbers. These numerical designations may encapsulate a diverse range of subjects, spanning from matters like politics and elections to sports and entertainment. 😊📊🌐🎉